In [1]:
import pandas as pd
import plotly.express as px
from fishtank.dimensions.spatial import (
    add_spatial_keys_to_facts,
    build_spatial_dimension_addition,
    append_spatial_dimension_addition,
    H3_RESOLUTIONS
)
from fishtank.db import get_engine

/var/folders/yr/sz_96ntj0d36ml4bwd_8by880000gn/T/ipykernel_87058/2101429849.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('data/gebco_2023_n78.9532_s34.4614_w160.5624_e237.5031.csv')
df

,lat,lon,elevation
0,34.488950,160.576073,-4065.805031
1,34.471461,160.649352,-4071.142857
2,34.453924,160.722620,-3835.073171
3,34.497825,160.777516,-4215.304167
4,34.436339,160.795875,-4055.071429
...,...,...,...
664334,78.986869,-145.419537,-3820.000000
664335,78.985730,-140.519530,-3783.000000
664336,78.983215,-136.050218,-3716.333333
664337,78.979901,-128.198033,-3271.000000


In [3]:
px.scatter_geo(df.sample(10000), lat='lat', lon='lon', color='elevation', projection='natural earth')

In [4]:
add_spatial_keys_to_facts(df, lon_col="lon", lat_col="lat")
df.head()

,lat,lon,elevation,h3_key_2,h3_key_4,h3_key_6
0,34.488950,160.576073,-4065.805031,586352508662710271,595359579068432383,604366775236165631
1,34.471461,160.649352,-4071.142857,586352508662710271,595359579068432383,604366776041471999
2,34.453924,160.722620,-3835.073171,586352508662710271,595359579068432383,604366774833512447
3,34.497825,160.777516,-4215.304167,586352508662710271,595359579068432383,604366774162423807
4,34.436339,160.795875,-4055.071429,586352508662710271,595359579068432383,604366774699294719


In [5]:
for resolution in H3_RESOLUTIONS:
    print(f"Building and Appending Addition for H3 resolution {resolution}...")
    dimension = build_spatial_dimension_addition(df, resolution)
    append_spatial_dimension_addition(dimension, resolution)

Building and Appending Addition for H3 resolution 2...
Building and Appending Addition for H3 resolution 4...
Building and Appending Addition for H3 resolution 6...


In [9]:
df.to_sql(
    "bathymetry",
    get_engine(),
    if_exists="append",
    index=False,
)

339